Username:Natee_ch  Rank 10
Nateethon chuleekran ID 6410422028 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
df_train = pd.read_csv('/kaggle/input/dads6003-in-class-competition/train.csv')
df_train.info()

ตรวจสอบข้อมูลพบว่า x หรือ features ทั้งหมดจากพบว่าเป็น numeric ทั้งหมด ลำดับต่อไปตรวจสอบ

In [ ]:
title=[]
for i in range(1,21):
    title.append("x"+str(i))
fig = make_subplots(rows=5, cols=4, start_cell="top-left",subplot_titles=title)
i=1
for r in range(1,6):
    for c in range(1,5):
        ft="x"+str(i)
        i=i+1
        fig.add_trace(go.Histogram(x=df_train[ft]),row=r, col=c)
#fig.add_trace(go.Histogram(x=df_train[ft]),row=(i//5)+1, col=i%4)
fig.show()


พบว่าข้อมูลกระจายตัวในลักษณะ normal distribution แต่มี scale ที่แตกต่างกันมาก ดังนั้นจึงเลือกที่จะ Standardization หรือ Z-Score Normalization

In [ ]:
# need Z-score nomorlize
from scipy.stats import zscore
df=df_train.copy()
df=df.dropna()
for x in df.columns:
    if x == "y":
        pass
    else:
        df[x]=(df[x]-df[x].mean())/df[x].std()


ลองหาว่า x correlation coefficient y มากที่สุด

In [ ]:
corrMatrix = df.corr()
xy_corr=corrMatrix['y'].abs().sort_values(ascending=False)
xy_corr=xy_corr.to_frame()
xy_corr=xy_corr.reset_index().loc[1::]
xy_corr=xy_corr.rename(columns={"index":"x","y":"correlation"})
xy_corr.head(20)
get_x=xy_corr["x"].to_list()

fig = px.bar(xy_corr, x='x', y='correlation')
fig.show()

In [ ]:
get_rank=get_x[0:13]
print(get_rank)
fig = px.scatter_3d(df, x='x2', y='x15', z='x3',
              color='y')
fig.show()

พบว่าข้อมูลมี X2 x15 x3 เป็นมี feature ที่มีสร้างความแตกต่างของ y 0 กับy1 ได้มากสุดตามลำดับ

In [ ]:
df2=df.dropna()
training_data = df2.sample(frac=0.8, random_state=25)
evaluate_data = df2.drop(training_data.index)

ตัดข้อมูลที่มี feature ไม่ครบออก และแบ่งข้อมูล  training_data และ evaluate_data

In [ ]:
# ทดลองใช้ NN
from sklearn.neural_network import MLPClassifier
df3=training_data.drop(columns=["y"])
X=df3[get_rank].to_numpy()
Y=training_data["y"].to_numpy()

mlp = MLPClassifier(hidden_layer_sizes=(40,5), max_iter=10000, alpha=1e-4,
                    solver='sgd', tol=1e-4, random_state=1,
                    learning_rate_init=.004,learning_rate='constant')

mlp.fit(X, Y)

df4=evaluate_data.drop(columns=["y"])
X_evaluate=df4[get_rank].to_numpy()
Y_evaluate=evaluate_data["y"].to_numpy()
print("NN")
print("Test set score: %f" % mlp.score(X_evaluate, Y_evaluate))
print("Training set score: %f" % mlp.score(X, Y))


In [ ]:
#ทดลองใช้ SVM
from sklearn import svm

clf_svm = svm.LinearSVC(dual=False)
clf_svm.fit(X, Y)

print("SVM")
print("Test set score: %f" % clf_svm.score(X_evaluate, Y_evaluate))
print("Training set score: %f" % clf_svm.score(X, Y))

In [ ]:
#ทดลองใช้
from sklearn.linear_model import LogisticRegression
clf_log = LogisticRegression(random_state = 0)
clf_log.fit(X, Y)

print("Logistic")
print("Test set score: %f" % clf_log.score(X_evaluate, Y_evaluate))
print("Training set score: %f" % clf_log.score(X, Y))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf_knn = KNeighborsClassifier(n_neighbors = 11, metric = 'minkowski', p = 2)
clf_knn.fit(X,Y)
print("knn")
print("Test set score: %f" % clf_knn.score(X_evaluate, Y_evaluate))
print("Training set score: %f" % clf_knn.score(X, Y))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
tree.fit(X, Y)
print("DecisionTree")
print("Test set score: %f" % tree.score(X_evaluate, Y_evaluate))
print("Training set score: %f" % tree.score(X, Y))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 19, criterion = 'entropy', random_state = 0)
forest.fit(X, Y)
print("RandomForest")
print("Test set score: %f" % forest.score(X_evaluate, Y_evaluate))
print("Training set score: %f" % forest.score(X, Y))

In [ ]:
from sklearn.metrics import confusion_matrix 
model=[mlp,clf_svm,clf_log,clf_knn,tree,forest]
name =["MLP","SVM","logistic","KNN","DecisionTree","RandomForest"] 
for i in range(len(model)):
    cm = confusion_matrix(Y_evaluate, model[i].predict(X_evaluate)) 
    #extracting TN, FP, FN, TP
    TN, FP, FN, TP = confusion_matrix(Y_evaluate, model[i].predict(X_evaluate)).ravel()
    print(cm)
    print('Model[{}] Testing Accuracy = "{} !"'.format(name[i],  (TP + TN) / (TP + TN + FN + FP)))
    precision=TP/(TP+FP)
    print('Model[{}] precision = "{} !"'.format(name[i], precision))
    Recall=TP/(TP+FN)
    print('Model[{}] Recall = "{} !"'.format(name[i],  Recall))
    F1=2*((precision*Recall)+(precision+Recall))
    print('Model[{}] F1-score = "{} !"'.format(name[i],F1))
    print()# Print a new line

ประเมิน Model ทั้งหมดพบว่า MLP ได้ ผลการประเมินดีสุด

In [ ]:
df_test = pd.read_csv('/kaggle/input/dads6003-in-class-competition/test.csv')
for x in df.columns:
    if x == "y":
        pass
    else:
        df_test[x]=(df_test[x]-df_train[x].mean())/df_train[x].std()

x_test=df_test[get_rank].to_numpy()
#out=clf.predict(x_test)
out=mlp.predict(x_test)
predict = out
fields = ['id','Expected']
df_out = pd.DataFrame()
id = list(range(1,2501))
df_out['id']=pd.Series(id)
df_out['Expected']=pd.Series(predict)
df_out.to_csv('submit19.csv', index=False)


In [ ]:
# Your work
# Must have
# 1. Cleansing data
# 2. Exploratory Data Analysis (describe insight and visualization)
# 3. Training data - Crossvalidation
# 4. Testing data